# Dekoratory

In [1]:
def shouter(func):
    def wrapper():
        print("Before", func.__name__)
        result = func()
        print("After", func.__name__)
        return result
    return wrapper

In [8]:
def hellome():
    print("Leszek")

hellome = shouter(hellome)

In [3]:
hellome()

Before hellome
Leszek
After hellome


In [5]:
@shouter
def hellome():
    print("Leszek")

In [6]:
hellome()

Before hellome
Leszek
After hellome


In [9]:
def hellome_without_decorator():
    '''asdf'''
    print("Leszek")
    
print(hellome_without_decorator.__doc__)

asdf


In [6]:
import functools

def shouter(func):
    @functools.wraps(func)
    def wrapper():
        print("Before", func.__name__)
        result = func()
        print("After", func.__name__)
        return result
    # wrapper.__doc__ = func.__doc__
    return wrapper

@shouter
@shouter
def hellome():
    '''asdf'''
    print("Leszek")
    
print(hellome.__doc__)

asdf


In [7]:
hellome.__name__

'hellome'

In [15]:
hellome()

Before hellome
Before hellome
Leszek
After hellome
After hellome


In [16]:
def dummy_decorator(func):
    return 42

@dummy_decorator
def foo():
    print('asdf')
    
foo

42

In [17]:
def assignment(func):
    return func()

@assignment
def foo():
    return 'asdf'
    
foo

'asdf'

In [19]:
sum = 5
print(sum)
del sum
print(sum)
del sum
print(sum)

5
<built-in function sum>


NameError: name 'sum' is not defined

In [20]:
from builtins import sum

In [22]:
import builtins

In [23]:
del builtins.sum

# Dekoratory parametryzowane

In [9]:
import functools

def tag(tagname):
    def decor(fun):
        @functools.wraps(fun)
        def wrapper(*args, **kwargs):
            print("<{0}>".format(tagname))
            ret = fun(*args, **kwargs)
            print("</{0}>".format(tagname))
            return ret
        return wrapper
    return decor

# decor = tag('b')
# output = decor(output)
@tag('b')
def output(data):
    print(data)
    
output('asdf')

<b>
asdf
</b>


# Dekorowanie klas

In [23]:
def addID(original_class):
    name = '_{}__id'.format(original_class.__name__)
    orig_init = original_class.__init__
    def __init__(self, *args, **kws):
        print("addID init")
        setattr(self, name, 123)
        orig_init(self, *args, **kws)

    original_class.__init__ = __init__
    return original_class

@addID
class Foo:
    def __init__(self):
        print("Foo class init")
        print(self.__id)
        
f = Foo()

addID init
Foo class init
123


In [24]:
f._Foo__id

123

# functools.partial

In [25]:
def foo(a, b, c):
    return a + b + c

In [26]:
from functools import partial

f = partial(foo, c=0)
f

functools.partial(<function foo at 0x7f52d80f5ea0>, c=0)

In [30]:
f(2, 3)

5

In [37]:
f = lambda *args, **kwargs: foo(*args, **kwargs, c=0)
f(2, 3)

5

In [29]:
# @partial(timeit, n=1)

In [39]:
l = [1, 2, 3, 4]
l[:6]

[1, 2, 3, 4]

In [40]:
from timeit import timeit

In [41]:
timeit??

In [44]:
def limit(n):
    def decor(fun):
        def wrapper(*args, **kwargs):
            out = fun(*args, **kwargs)
            return out[:n]
        return wrapper
    return decor

In [45]:
import collections

In [46]:
g = (i for i in range(5))
isinstance(g, collections.Generator)

True

In [48]:
isinstance([], collections.Sequence)

True

In [52]:
from itertools import islice

def limit(n):
    def decor(fun):
        def wrapper(*args, **kwargs):
            out = fun(*args, **kwargs)
            try:
                return out[:n]
            except TypeError:
                return islice(out, n)
        return wrapper
    return decor

In [54]:
@limit(5)
def foo():
    return (i for i in range(100))

list(foo())

[0, 1, 2, 3, 4]

In [60]:
import itertools

@limit(5)
def foo():
    yield 'asdf'
    yield from itertools.count(1)

foo()

# Klasy jako dekoratory

In [61]:
class shout:
    def __init__(self, f):
        print("inside decorator's __init__()")
        self.f = f

    def __call__(self):
        print("before call")
        self.f()
        print("after call")

In [62]:
@shout
def function():
    print('inside')
    
# function = shout(function)

inside decorator's __init__()


In [63]:
function()

before call
inside
after call


In [64]:
bool.__mro__

(bool, int, object)

In [65]:
hash(True)

1

In [66]:
hash(1)

1

In [77]:
hash('a')

-9197565182068217378

In [78]:
hash('ą')

7040219612788072914

In [75]:
hash('ą'.encode('utf-8'))

2025755770517931098

# Menedżery kontekstu

```python
with VAR = EXPR:
    BLOCK
```

jest równoważna:

```python
VAR = EXPR
VAR.__enter__()
try:
    BLOCK
finally:
    VAR.__exit__()
```

In [79]:
s = open('README.md')
try:
    print(len(s.read()))
finally:
    s.close()

143


In [80]:
with open('README.md') as s:
    print(len(s.read()))

143


In [95]:
class Context:
    def __init__(self):
        print('__init__()')

    def __enter__(self):
        print('__enter__()')
        return 42

    def __exit__(self, exc_type, exc_val, exc_tb):
        print('__exit__()')
        print('exc_type = ', exc_type)
        print('exc_val = ', exc_val)
        print('exc_tb = ', exc_tb)
        return True  # rzucony wyjątek nie jest propagowany dalej

In [96]:
c = Context()
print('before with')
with c as cc:
    print('doing work in the context')
    print('c = ', c)
    raise KeyError('msg')
    print('cc = ', cc)
print('after with')

__init__()
before with
__enter__()
doing work in the context
c =  <__main__.Context object at 0x7f52d816db70>
__exit__()
exc_type =  <class 'KeyError'>
exc_val =  'msg'
exc_tb =  <traceback object at 0x7f52d8180b08>
after with


In [97]:
class Context:
    def __init__(self):
        print('__init__()')

    def __enter__(self):
        print('__enter__()')
        return 42

    def __exit__(self, exc_type, exc_val, exc_tb):
        print('__exit__()')
        
c = Context()  # c = Context.__init__()
print('before with')
with c as cc:  # cc = c.__enter__()
    print('doing work in the context')
    print('c = ', c)
    print('cc = ', cc)
print('after with')

__init__()
before with
__enter__()
doing work in the context
c =  <__main__.Context object at 0x7f52d8139048>
cc =  42
__exit__()
after with


# `@contextmanager`

In [105]:
from contextlib import contextmanager

@contextmanager
def Shouter():
    print('begin')
    try:
        yield 42
    except Exception:
        print('exception')
    print('end')

In [106]:
with Shouter() as c:
    print('inside')
    print(c)

begin
inside
42
end


In [103]:
with Shouter():
    print('inside')
    raise Exception
    print('inside end')

begin
inside
exception
end
